# Mapping 3
- Line of Interest (circle & straight line)
- Frequency spectrum mapping
- Linear interpolation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches
from IPython.display import Audio
from scipy.io.wavfile import write
from datetime import datetime
import subprocess
from pya import *
from scipy.interpolate import LinearNDInterpolator

%matplotlib widget

In [ ]:
device_info();

In [ ]:
s = startup()

In [ ]:
s

In [ ]:
#shutdown()

# Simulation

In [ ]:
import simulation

n = 128
sim_speed = 0.004
sim_fps = 400
duration = 4
frame_amount = duration * sim_fps

initial_state = np.array([[simulation.gaussian(x, y, n, offset=[-0.6, 0.0], width=0.15) for x in range(n)] for y in range(n)])
potential = np.array([[simulation.parabola(x, y, n, offset=[0, 0], factor=10000) for x in range(n)] for y in range(n)])

multi_slit = [(-15, -13), (-8, -6), (-1, 1), (6, 8), (13, 15)]
double_slit = [(-4, -2), (2, 4)]
single_slit = [(-2, 2)]
slits = double_slit

frames = simulation.sim(n, sim_fps, duration, slits, sim_speed, initial_state=initial_state, potential=potential)

In [ ]:
import matplotlib
plt.figure()
plt.pcolormesh(np.abs(frames[0]) ** 2, cmap='inferno', norm=matplotlib.colors.PowerNorm(vmin=0, vmax=np.max(np.square(np.abs(frames))), gamma=0.4))
# plt.pcolormesh(potential, vmin=0, vmax=20000)
plt.colorbar()
plt.show()

# Sonification

In [ ]:
# circle of interest
def circle(radius, center):
    radians = np.linspace(0, 2 * np.pi, num_frequencies, endpoint=False)
    x = (0.5 * n * (radius * np.cos(radians) + 1 + center[0]))
    y = (0.5 * n * (radius * np.sin(radians) + 1 + center[1]))
    return x, y

# line of interest
def line(start, end):
    t = np.linspace(0, 1, num_frequencies, endpoint=False)
    x = start[0]+n//2 + t * (end[0] - start[0])
    y = start[1]+n//2 + t * (end[1] - start[1])
    return x, y

In [ ]:
import numexpr as ne

def interpolate(indices, x_vals, y_vals):
    # Hardcoded interpolation
    # Way faster than any other interpolation!
    t = np.array([indices, y_vals, x_vals]) % 1
    #return t
    floors = np.floor([indices, y_vals, x_vals]).astype(int)
    ceils  = np.ceil ([indices, y_vals, x_vals]).astype(int)
    
    return  ((1-t[0]) * ((1-t[1]) * ((1-t[2]) * np.square(np.abs(frames[floors[0], floors[1], floors[2]]))
                                  +     t[2]  * np.square(np.abs(frames[floors[0], floors[1], ceils [2]])))
                      +     t[1]  * ((1-t[2]) * np.square(np.abs(frames[floors[0], ceils [1], floors[2]]))
                                  +     t[2]  * np.square(np.abs(frames[floors[0], ceils [1], ceils [2]]))))
              + t[0]  * ((1-t[1]) * ((1-t[2]) * np.square(np.abs(frames[ceils [0], floors[1], floors[2]]))
                                  +     t[2]  * np.square(np.abs(frames[ceils [0], floors[1], ceils [2]])))
                      +     t[1]  * ((1-t[2]) * np.square(np.abs(frames[ceils [0], ceils [1], floors[2]]))
                                  +     t[2]  * np.square(np.abs(frames[ceils [0], ceils [1], ceils [2]])))))

In [ ]:
# parameters
sample_rate = 44100
num_frequencies = 200

f_min = 100
f_max = 4000

# choose method
x, y = circle(radius=0.6, center=[0, 0])
# x, y = line(start=[-60, 0], end=[60, 0])

In [ ]:
# Sonification
frames_indices = np.linspace(0, frame_amount - 1, sample_rate * duration, endpoint=False)

frequencies = np.linspace(f_min, f_max, num_frequencies, endpoint=False)
# frequencies = np.fft.fftfreq(num_frequencies * 2, 1 / sample_rate)[:num_frequencies] / sample_rate*2 * (f_max - f_min) + f_min

t = np.arange(sample_rate * duration) / sample_rate
sini = np.sin(2 * np.pi * frequencies[np.newaxis, :] * t[:, np.newaxis])

f1 = frames_indices[:, np.newaxis].repeat(num_frequencies, axis=1).flatten()
x1 = x[np.newaxis, :].repeat(sample_rate * duration, axis=0).flatten()
y1 = y[np.newaxis, :].repeat(sample_rate * duration, axis=0).flatten()

In [ ]:
# plt.figure()
# plt.plot(frequencies)

In [ ]:
# action
audio = interpolate(f1, x1, y1)
audio = np.sum(sini * audio.reshape(sample_rate * duration, num_frequencies), axis=1)

In [ ]:
pa = Asig(audio, sr=sample_rate)

In [ ]:
# save WAV
audio_filename = f'output/sonification_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}.wav'
pa.save_wavfile(audio_filename)
print(f"Sonification saved as {audio_filename}")

In [ ]:
# play directly
pa.fade_in(0.005).fade_out(0.005).stereo().play(onset=1, rate=1)

In [ ]:
plt.figure()
pa.plot()

# Video

In [ ]:
import video

# save video
video_filename, anim = video.create(frames, 20, frame_amount, sim_fps, slits, n)

#plt.close()

In [ ]:
# combine audio & video
video.combine(audio_filename, video_filename)